# Interactive Commute Time Notebook
### Author: Kenny Mai
### Original Author: Jim Maguire
### Last Modified: 7/19/23
### Required Files: chromedriver, scholar_list.sql, prod_scholar_commute_times.csv
This notebook will walk through multiple iterations of commute times with human supervision. Every loop, the commute times that are found will be added to the master list and removed from the list to be scraped. 

### Packages, dictionaries, and initializing the web scraper

In [ ]:
# Import relevant packages
import csv
import pandas as pd
import numpy as np
import datetime
import time
from aws_helper_functions import aws_helper_functions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from datetime import date
from time import sleep

In [ ]:
# Define school names
schools = {
    'SA-BS': '211 Throop Avenue, Floor 3, Brooklyn, NY 11206',
    'SA-BS1': '70 Tompkins Ave, Brooklyn, NY 11206',
    'SA-BS2': '211 Throop Avenue, Floor 3, Brooklyn, NY 11206',
    'SA-BSMS': '70 Tompkins Avenue, Floor 2, Brooklyn, NY 11206',
    'SA-BH': '99 Avenue P, Floor 4, Brooklyn, NY 11204',
    'SA-BB':'1420 East 68th Street, Floor 3, Brooklyn, NY 11234',
    'SA-BX1': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'SA-BX1MS': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'SA-BX2': '450 St. Pauls Place, Floor 5, Bronx, NY 10456',
    'SA-BX2MS': '270 East 167 Street, Floor 2, Bronx, NY 10456',
    'SA-BX3': '1000 Teller Avenue, Floor 4, Bronx, NY 10456',
    'SA-BX4': '885 Bolton Avenue, Floor 3, Bronx, NY 10473',
    'SA-BX5': '604 East 139th Street, Bronx, NY 10454',
    'SA-BXMS': '965 Longwood Avenue, Floor 2, Bronx, NY 10459',
    'SA-BW': '139 Menahan Street, Brooklyn, NY 11221',
    'SA-CH': '284 Baltic Street, Lower Level, Brooklyn, NY 11201',
    'SA-CR': '330 Crown Street, Floor 5, Brooklyn, NY 11225',
    'SA-DPMS': '72 Veronica Place, Floor 3, Brooklyn, NY 11226',
    'SA-EFMS': '3109 Newkirk Avenue, Brooklyn, NY, 11226',
    'SA-FR': '10-45 Nameoke St, Floor 3, Queens, NY 11691',
    'SA-FRMS': '10-45 Nameoke St, Queens, NY 11691',
    'SA-FB': '15 Snyder Avenue, Brooklyn, NY 11226',
    'SA-HHMS': '461 West 131st Street, New York, NY 10027',
    'SA-H1': '34 West 118th Street, Floor 2, New York, NY 10026',
    'SA-H2': '144 East 128th Street, Floor 3, New York, NY 10035',
    'SA-H3': '410 East 100th Street, Floor 4, New York, NY 10029',
    'SA-H4': '240 West 113th Street, Floor 3, New York, NY 10026',
    'SA-H5': '301 West 140th Street, Floor 3, New York, NY 10030',
    'SA-H6': '461 West 131st Street, New York, NY 10027',
    'SA-HE': '141 East 111th Street, Floor 3, New York, NY 10029',
    'SA-HNC': '175 West 134th Street, Floor 3, New York, NY 10030',
    'SA-HW': '215 West 114th Street, Floor 5, New York, NY 10026',
    'SA-HK': '439 West 49th Street, Floor 2, New York NY 10019',
    'SA-HSLA-HA': '509 West 129th Street, Floor 3,New York, NY 10027',
    'SA-HSLA-MA': '111 East 33rd Street, 4th Floor, New York, NY 10016',
    'SA-HSLA-BK': '70 Tompkins Avenue, 2nd Fl., Brooklyn, NY 11206',
    'SA-HY': '500 West 41st Street, New York, NY 10018',
    'SA-HYMS': '500 West 41st Street, New York, NY 10018',
    'SA-LAMS': '787 Lafayette Avenue, Brooklyn, NY 11221',
    'SA-MWMS': '439 West 49th Street, New York, NY 10019',
    'SA-MYMS': '700 Park Avenue, Floor 3, Brooklyn, NY 11206',
    'SA-NW': '268 E. 207th Street, Bronx, NY 10467',
    'SA-OZMS': '109-55 128th Street South, Queens, NY 11420',
    'SA-PH': '801 Park Place, Floor 4, Brooklyn, NY 11216',
    'SA-QV': '92-53 Springfield Boulevard, Queens, NY 11428',
    'SA-QV1': '92-53 Springfield Boulevard, Queens, NY 11428',
    'SA-QV2': '92-53 Springfield Boulevard, Queens, NY 11428',
    'SA-RD': '133-25 Guy R Brewer Blvd, Jamaica, NY, 11434',
    'SA-RO': '147-65 249th Street, Rosedale, NY 11422',
    'SA-SB': '3000 Avenue X, Brooklyn, NY 11235',
    'SA-SJ': '120-27 141st Street, Jamaica, NY 11436',
    'SA-SG': '132-55 Ridgedale Street, Floor 3, Queens, NY 11413',
    'SA-SGMS': '132-55 Ridgedale St, Queens, NY 11413',
    'SA-SGS': '143-10 Springfield Blvd, Springfield Gardens, NY, 11413',
    'SA-US': '40 Irving Place, Floor 2, New York, NY 10003',
    'SA-UW': '145 West 84th Street, Floor 2, New York, NY 10024',
    'SA-WH': '701 Fort Washington Avenue, New York, NY 10040',
    'SA-WB': '183 South 3rd Street, Floor 4, Brooklyn, NY 11211',
    'SA-FG': '101 Park Ave, Brooklyn, NY 11205',
    'Bed-Stuy': '211 Throop Avenue, Floor 3, Brooklyn, NY 11206',
    'Bed-Stuy 2': '211 Throop Avenue, Floor 3, Brooklyn, NY 11206',
    'Bed-Stuy Middle School': '70 Tompkins Avenue, Floor 2, Brooklyn, NY 11206',
    'Bensonhurst': '99 Avenue P, Floor 4, Brooklyn, NY 11204',
    'Bergen Beach':'1420 East 68th Street, Floor 3, Brooklyn, NY 11234',
    'Bronx 1': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'Bronx 1 Middle School': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'Bronx 2': '450 St. Pauls Place, Floor 5, Bronx, NY 10456',
    'Bronx 2 Middle School': '270 East 167 Street, Floor 2, Bronx, NY 10456',
    'Bronx 3': '1000 Teller Avenue, Floor 4, Bronx, NY 10456',
    'Bronx 4': '885 Bolton Avenue, Floor 3, Bronx, NY 10473',
    'Bronx 5': '604 East 139th Street, Bronx, NY 10454',
    'Bronx 5 Lower': '604 East 139th Street, Bronx, NY 10454',
    'Bronx 5 Upper': '604 East 139th Street, Bronx, NY 10454',
    'Bronx Middle School': '965 Longwood Avenue, Floor 2, Bronx, NY 10459',
    'Bushwick': '139 Menahan Street, Brooklyn, NY 11221',
    'Cobble Hill': '284 Baltic Street, Lower Level, Brooklyn, NY 11201',
    'Crown Heights': '330 Crown Street, Floor 5, Brooklyn, NY 11225',
    'Ditmas Park Middle School': '72 Veronica Place, Floor 3, Brooklyn, NY 11226',
    'East Flatbush Middle School': '3109 Newkirk Avenue, Brooklyn, NY, 11226',
    'Far Rockaway': '10-45 Nameoke St, Floor 3, Queens, NY 11691',
    'Far Rockaway Middle School': '10-45 Nameoke St, Queens, NY 11691',
    'Flatbush': '15 Snyder Avenue, Brooklyn, NY 11226',
    'Hamilton Heights Middle School': '461 West 131st Street, New York, NY 10027',
    'Harlem 1': '34 West 118th Street, Floor 2, New York, NY 10026',
    'Harlem 2': '144 East 128th Street, Floor 3, New York, NY 10035',
    'Harlem 3': '410 East 100th Street, Floor 4, New York, NY 10029',
    'Harlem 4': '240 West 113th Street, Floor 3, New York, NY 10026',
    'Harlem 5': '301 West 140th Street, Floor 3, New York, NY 10030',
    'Harlem 6': '461 West 131st Street, New York, NY 10027',
    'Harlem East': '141 East 111th Street, Floor 3, New York, NY 10029',
    'Harlem North Central': '175 West 134th Street, Floor 3, New York, NY 10030',
    'Harlem West': '215 West 114th Street, Floor 5, New York, NY 10026',
    'Hells Kitchen': '439 West 49th Street, Floor 2, New York NY 10019',
    'Hell\'s Kitchen': '439 West 49th Street, Floor 2, New York NY 10019',
    'High School of the Liberal Arts - Harlem': '509 West 129th Street, Floor 3,New York, NY 10027',
    'High School of the Liberal Arts-Harlem': '509 West 129th Street, Floor 3,New York, NY 10027',
    'High School of the Liberal Arts - Manhattan': '111 East 33rd Street, 4th Floor, New York, NY 10016',
    'High School of the Liberal Arts-Manhattan': '111 East 33rd Street, 4th Floor, New York, NY 10016',
    'High School of the Liberal Arts - Brooklyn': '70 Tompkins Avenue, 2nd Fl., Brooklyn, NY 11206',
    'High School of the Liberal Arts-Brooklyn': '70 Tompkins Avenue, 2nd Fl., Brooklyn, NY 11206',
    'Hudson Yards': '500 West 41st Street, New York, NY 10018',
    'Hudson Yards Middle School': '500 West 41st Street, New York, NY 10018',
    'Lafayette Middle School': '787 Lafayette Avenue, Brooklyn, NY 11221',
    'Midtown West Middle School': '439 West 49th Street, New York, NY 10019',
    'Myrtle Middle School': '700 Park Avenue, Floor 3, Brooklyn, NY 11206',
    'Norwood': '268 E. 207th Street, Bronx, NY 10467',
    'Ozone Park Middle School': '109-55 128th Street South, Queens, NY 11420',
    'Prospect Heights': '801 Park Place, Floor 4, Brooklyn, NY 11216',
    'Queens Village': '92-53 Springfield Boulevard, Queens, NY 11428',
    'Rochdale': '133-25 Guy R Brewer Blvd, Jamaica, NY, 11434',
    'Rosedale': '147-65 249th Street, Rosedale, NY 11422',
    'Sheepshead Bay': '3000 Avenue X, Brooklyn, NY 11235',
    'South Jamaica': '120-27 141st Street, Jamaica, NY 11436',
    'Springfield Gardens': '132-55 Ridgedale Street, Floor 3, Queens, NY 11413',
    'Springfield Gardens Middle School': '132-55 Ridgedale St, Queens, NY 11413',
    'Springfield Gardens MS': '132-55 Ridgedale St, Queens, NY 11413',
    'Springfield Gardens South': '143-10 Springfield Blvd, Springfield Gardens, NY, 11413',
    'Union Square': '40 Irving Place, Floor 2, New York, NY 10003',
    'Upper West': '145 West 84th Street, Floor 2, New York, NY 10024',
    'Washington Heights': '701 Fort Washington Avenue, New York, NY 10040',
    'Williamsburg': '183 South 3rd Street, Floor 4, Brooklyn, NY 11211',
    'SA Bed-Stuy': '211 Throop Avenue, Floor 3, Brooklyn, NY 11206',
    'SA Bergen Beach': '1420 East 68th Street, Floor 3, Brooklyn, NY 11234',
    'SA Bronx 1': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'SA Bronx 1 Middle School': '339 Morris Avenue, Floor 2, Bronx, NY 10451',
    'SA Bronx 2': '450 St. Pauls Place, Floor 5, Bronx, NY 10456',
    'SA Bronx 3': '1000 Teller Avenue, Floor 4, Bronx, NY 10456',
    'SA Bronx 4': '885 Bolton Avenue, Floor 3, Bronx, NY 10473',
    'SA Bronx 5': '604 East 139th Street, Bronx, NY 10454',
    'SA Bushwick': '139 Menahan Street, Brooklyn, NY 11221',
    'SA Cobble Hill': '284 Baltic Street, Lower Level, Brooklyn, NY 11201',
    'SA Crown Heights': '330 Crown Street, Floor 5, Brooklyn, NY 11225',
    'SA Ditmas Park Middle School': '72 Veronica Place, Floor 3, Brooklyn, NY 11226',
    'SA Far Rockaway': '10-45 Nameoke St, Floor 3, Queens, NY 11691',
    'SA Flatbush': '15 Snyder Avenue, Brooklyn, NY 11226',
    'SA Hamilton Heights Middle School': '461 West 131st Street, New York, NY 10027',
    'SA Harlem 1': '34 West 118th Street, Floor 2, New York, NY 10026',
    'SA Harlem 2': '144 East 128th Street, Floor 3, New York, NY 10035',
    'SA Harlem 3': '410 East 100th Street, Floor 4, New York, NY 10029',
    'SA Harlem 4': '240 West 113th Street, Floor 3, New York, NY 10026',
    'SA Harlem 5': '301 West 140th Street, Floor 3, New York, NY 10030',
    'SA Harlem East': '141 East 111th Street, Floor 3, New York, NY 10029',
    "SA Hell's Kitchen": '439 West 49th Street, Floor 2, New York NY 10019',
    'SA Hudson Yards': '500 West 41st Street, New York, NY 10018',
    'SA Norwood': '268 E. 207th Street, Bronx, NY 10467',
    'SA Ozone Park Middle School': '109-55 128th Street South, Queens, NY 11420',
    'SA Prospect Heights': '801 Park Place, Floor 4, Brooklyn, NY 11216',
    'SA Queens Village': '92-53 Springfield Boulevard, Queens, NY 11428',
    'SA Rosedale': '147-65 249th Street, Rosedale, NY 11422',
    'SA Sheepshead Bay': '3000 Avenue X, Brooklyn, NY 11235',
    'SA South Jamaica': '120-27 141st Street, Jamaica, NY 11436',
    'SA Union Square': '40 Irving Place, Floor 2, New York, NY 10003',
    'SA Upper West': '145 West 84th Street, Floor 2, New York, NY 10024',
    'SA Washington Heights': '701 Fort Washington Avenue, New York, NY 10040',
    'SA Williamsburg': '183 South 3rd Street, Floor 4, Brooklyn, NY 11211'
    }

In [ ]:
# Driver Setup and getting the Google Maps page to set Arrive By time to 8:00 AM on Monday October 17th 
driver = webdriver.Chrome()
CHROMEDRIVER_PATH = Service(executable_path='chromedriver')
options = Options()
#options.headless = True
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=CHROMEDRIVER_PATH, options=options)
driver.get('https://www.google.com/maps/dir///@40.7422956,-73.9891463,13z')
driver.implicitly_wait(15)
start_box = driver.find_element(By.XPATH, '//*[@id="sb_ifc50"]/input')
dest_box = driver.find_element(By.XPATH, '//*[@id="sb_ifc51"]/input')
start_box.send_keys('Pennsylvania Station, New York, NY 10119')
dest_box.send_keys('95 Pine Street Success Academy')

driver.find_element(By.XPATH, '//*[@id="directions-searchbox-1"]/button[1]').click()  # click search button
driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/div/div/div/div[2]').click()  # click "Leave Now/Arrive By" dropdown
driver.find_element(By.XPATH, '//*[@id=":2"]/div').click()  # click Arrive By

time_box = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/span[1]/input')
time_box.send_keys(Keys.CONTROL + 'a')
time_box.send_keys(Keys.DELETE)
time_box.send_keys('8:00 AM')

# Loop Start

### Load data

In [ ]:
# This is the population of current scholars whose commute times need to be added to the existing file
query_file = 'scholar_list.sql'
query = open(query_file).read()
df_missing_scholars_raw = aws_helper_functions.read_from_redshift(query, local_mode=True)
# This is the population of applicants with offers whos commute times need to be added to the existing file
query_file = 'offer_list.sql'
query = open(query_file).read()
df_new_offers_raw = aws_helper_functions.read_from_redshift(query, local_mode=True)
# Existing commute times file
df_commute_times_raw = pd.read_csv('raw_data_science.raw_scholar_commute_times.csv')
print('Missing scholar population: ',df_missing_scholars_raw.shape)
print('df_missing_scholars columns: ',df_missing_scholars_raw.columns)
print('New offer population: ',df_new_offers_raw.shape)
print('df_new_offers columns: ',df_new_offers_raw.columns)
print('Existing scholars with commutes: ',df_commute_times_raw.shape)
print('df_commute_times columns: ',df_commute_times_raw.columns)

### Data prep

In [ ]:
# Checkpoint
df_commute_times = df_commute_times_raw
df_nmissing_scholars = df_missing_scholars_raw
# Removing scholars whose scraping failed last run
old_targets = df_commute_times[(df_commute_times['driving_time'].isna()==True) & (df_commute_times['transit_time'].isna()==True) & (df_commute_times['walking_time'].isna()==True)]
df_commute_times.drop(old_targets.index, inplace=True)
# Prepping existing scholars
df_missing_scholars = df_missing_scholars_raw[['sa_scholar_id','school_name','address']]
df_all = df_missing_scholars.merge(df_commute_times.drop_duplicates(), on=['sa_scholar_id'], how='left', indicator=True)
df_scholar_list = df_all[df_all['_merge']=='left_only']
df_scholar_list['street'] = df_scholar_list['address'].str.extract('([^#]+)')
df_scholar_list['street'] = df_scholar_list['street'].str.extract('([^,]+)')
df_scholar_list['address'] = df_scholar_list['street'] + df_scholar_list['address'].str[-7:]
df_scholar_list = df_scholar_list[['sa_scholar_id','school_name','address']]
df_scholar_list.drop_duplicates(subset=['sa_scholar_id'],inplace=True)
df_scholar_list

In [ ]:
# Check for missing schools
for school in df_scholar_list['school_name']:
    if school not in schools:
        print(school)

# Clean up bad addresses by hand
This is the step requiring human supervision. The addresses are entered manually and are sometimes incorrect to the point that Google Maps cannot find the address. 

In [ ]:
# Cleaning up the bad addresses by hand. Export first, edit the csv
df_scholar_list.to_csv('cleanup_step.csv',index=False)

# Upload corrected addresses back

In [ ]:
# Now read in and overwrite with the hand-corrected addresses
df_scholar_list = pd.read_csv('cleanup_step.csv')

# Scraping

In [ ]:
# Scraping loop
missing_commute_times = pd.DataFrame(columns = ['sa_scholar_id','Driving','Transit','Walking'])
for index, row in df_scholar_list.iterrows():
    print(index, row['sa_scholar_id'])
    start_box.send_keys(Keys.CONTROL + 'a')
    start_box.send_keys(Keys.DELETE)
    start_box.send_keys(row['address'])

    sleep(0.5)
    dest_box.send_keys(Keys.CONTROL + 'a')
    dest_box.send_keys(Keys.DELETE)
    dest_box.send_keys(schools[row['school_name']])
    dest_box.send_keys(Keys.ENTER)
    driver.implicitly_wait(1)
    sleep(1.5)

    try:
        old_mode_1 = 'Driving'
        old_time_1 = driver.find_element(By.XPATH, '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[2]/button/div[1]').text.replace('\n', '')
    except:
        old_mode_1 = 'NA'
        old_time_1 = 'NA'
    try:
        old_mode_2 = 'Transit'
        old_time_2 = driver.find_element(By.XPATH, '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[3]/button/div[1]').text.replace('\n', '')
    except:
        old_mode_2 = 'NA'
        old_time_2 = 'NA'
    try:
        old_mode_3 = 'Walking'
        old_time_3 = driver.find_element(By.XPATH, '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[4]/button/div[1]').text.replace('\n', '')
    except:
        old_mode_3 = 'NA'
        old_time_3 = 'NA'
    missing_commute_times.loc[len(missing_commute_times)] = [row['sa_scholar_id'],old_time_1,old_time_2,old_time_3]

# Cleaning up the results to align with the existing data

In [ ]:
# I hate loops. I'm so sorry. -Kenny
cleaned = pd.DataFrame(missing_commute_times)
cleaned['driving_time'] = (pd.to_numeric(np.where(cleaned['Driving'].str.contains('hr'),cleaned['Driving'].str.split(' ').str[0],0))*3600)+(pd.to_numeric(np.where(cleaned['Driving'].str.contains('hr'),cleaned['Driving'].str.split(' ').str[2],0))*60)
cleaned['driving_time'] = (pd.to_numeric(np.where(cleaned['Driving'].str.contains('hr'),cleaned['driving_time'],pd.to_numeric(cleaned['Driving'].str.split(' ').str[0])*60)))
cleaned['transit_time'] = (pd.to_numeric(np.where(cleaned['Transit'].str.contains('hr'),cleaned['Transit'].str.split(' ').str[0],0))*3600)+(pd.to_numeric(np.where(cleaned['Transit'].str.contains('hr'),cleaned['Transit'].str.split(' ').str[2],0))*60)
cleaned['transit_time'] = (pd.to_numeric(np.where(cleaned['Transit'].str.contains('hr'),cleaned['transit_time'],pd.to_numeric(cleaned['Transit'].str.split(' ').str[0])*60)))
cleaned['walking_time'] = (pd.to_numeric(np.where(cleaned['Walking'].str.contains('hr'),cleaned['Walking'].str.split(' ').str[0],0))*3600)+(pd.to_numeric(np.where(cleaned['Walking'].str.contains('hr'),cleaned['Walking'].str.split(' ').str[2],0))*60)
cleaned['walking_time'] = (pd.to_numeric(np.where(cleaned['Walking'].str.contains('hr'),cleaned['walking_time'],pd.to_numeric(cleaned['Walking'].str.split(' ').str[0])*60)))
cleaned = cleaned.drop(['Driving','Transit','Walking'],axis=1)
cleaned['last_run_date'] = date.today()
cleaned

In [ ]:
# Cleanup incase it merges an extra column because I'm careless with my code. -Kenny
df_commute_times['last_run_date'] = df_commute_times['last_run_date'].combine_first(df_commute_times['last_run_date_x'])
df_commute_times = df_commute_times.drop(['last_run_date_x'],axis=1)
df_commute_times

### Final merge

In [ ]:

prod_scholar_commute_times = df_commute_times.merge(cleaned,how='outer',on=['sa_scholar_id','driving_time','transit_time','walking_time','last_run_date'])
prod_scholar_commute_times

In [ ]:
prod_scholar_commute_times.to_csv('raw_data_science.raw_scholar_commute_times.csv',index=False)

# Loop End